<a href="https://colab.research.google.com/github/fenwickslab/fenwicks/blob/master/tutorials/tutorial5_flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -qq -U cufflinks

In [0]:
import tensorflow as tf
import os
import numpy as np

In [3]:
if tf.io.gfile.exists('./fenwicks'):
  tf.io.gfile.rmtree('./fenwicks')
!git clone -q https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw

In [4]:
ROOT_DIR = 'gs://gs_colab'
PROJECT = 'tutorial5'
MODEL = "ResNet50" #@param ["InceptionResNetV2", "ResNet50", "ResNet50V2", "InceptionV3", "MobileNetV2", "Xception"]

BATCH_SIZE = 128 #@param ["64", "128", "256", "512"] {type:"raw"}
EPOCHS = 11 #@param {type:"slider", min:1, max:100, step:1}
LEARNING_RATE = 0.001 #@param ["0.001", "0.01"] {type:"raw"}
WARMUP = 0.05 #@param {type:"slider", min:0, max:0.5, step:0.05}

In [5]:
fw.colab_utils.setup_gcs()

In [6]:
data_dir, work_dir = fw.io.get_project_dirs(ROOT_DIR, PROJECT)

In [7]:
base_model = fw.keras_models.get_model(MODEL, ROOT_DIR)

In [8]:
data_dir_local = fw.datasets.untar_data(fw.datasets.URLs.FLOWER_PHOTOS, './flower_photos')

In [9]:
data_dir_local = os.path.join(data_dir_local, 'flower_photos')
data_dir_local

'./flower_photos/datasets/flower_photos'

In [10]:
data_fn = os.path.join(data_dir, 'all.tfrecord')

paths, y, labels = fw.data.data_dir_tfrecord(data_dir_local, data_fn, shuffle=True)
n_classes = len(labels)
n_all = len(y)

INFO:tensorflow:Output file already exists. Skipping.


In [11]:
fw.plt.plot_counts_pie(y, labels)

In [12]:
fw.anim.show_image_files(paths)

In [13]:
def build_nn():
  base = base_model.model_func()
  base.trainable = False
  model = fw.Sequential()
  model.add(base)
  model.add(tf.keras.layers.Flatten())
  model.add(fw.layers.Classifier(n_classes))
  return model

In [14]:
n_valid = n_all // 5 // 8 * 8
n_train = n_all - n_valid
steps_per_epoch = n_train // BATCH_SIZE
total_steps = steps_per_epoch * EPOCHS
warmup_steps = int(total_steps * WARMUP)

In [15]:
cosine_decay = tf.train.cosine_decay_restarts
lr_func = fw.train.one_cycle_lr(LEARNING_RATE, total_steps, warmup_steps, cosine_decay)
fw.plt.plot_lr_func(lr_func, total_steps)

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
opt_func = fw.train.adam_optimizer(lr_func)
model_func = fw.train.get_clf_model_func(build_nn, opt_func)

In [17]:
def get_input_parser(training):
  h = w = base_model.img_size
  tfms = fw.transform.get_fastai_transforms(h, w, training=training, normalizer=base_model.normalizer)
  return fw.data.get_tfexample_image_parser(tfms)

parser_train = get_input_parser(True)
parser_eval = get_input_parser(False)

In [18]:
est = fw.train.get_tpu_estimator(n_train // BATCH_SIZE, model_func, work_dir, 
  base_model.weight_dir, base_model.weight_vars, trn_bs=BATCH_SIZE, val_bs=n_valid)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/tutorial5/2019-05-16-15:44:07', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.73.202.122:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fded79ab208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.73.202.122:8470', '_evaluation_master': 'grpc://10.73.202.122:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=22, num_shards=N

In [19]:
def train_eval_fold(val_fold):
  train_input_func = lambda params: fw.data.tfrecord_ds(data_fn, parser_train,
    params['batch_size'], n_folds=5, val_fold_idx = val_fold, training=True)
  valid_input_func = lambda params: fw.data.tfrecord_ds(data_fn, parser_eval, 
    params['batch_size'], n_folds=5, val_fold_idx = val_fold, training=False)

  est.train(train_input_func, steps=total_steps)
  result = est.evaluate(valid_input_func, steps=1)
  fw.io.create_clean_dir(work_dir)
  return result

In [20]:
result = []
result.append(train_eval_fold(0))

INFO:tensorflow:Querying Tensorflow master (grpc://10.73.202.122:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 11018054164415147342)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 14716372907037514274)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7087530949167762374)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 6730544847693643781)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 10607720766597758090)
INFO:tensorflow:*** Available Device: _DeviceAttribute

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning:

The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.



Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[

In [21]:
result.append(train_eval_fold(1))

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning:

The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.



INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[^/]', 'res4a_branch2a/bias[^/]', 'res4a_branch1/

In [22]:
result.append(train_eval_fold(2))

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning:

The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.



INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[^/]', 'res4a_branch2a/bias[^/]', 'res4a_branch1/

In [23]:
result.append(train_eval_fold(3))

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning:

The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.



INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[^/]', 'res4a_branch2a/bias[^/]', 'res4a_branch1/

In [24]:
result.append(train_eval_fold(4))

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning:

The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.



INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[^/]', 'res4a_branch2a/bias[^/]', 'res4a_branch1/

In [25]:
acc = [res['accuracy'] for res in result]
loss = [res['loss'] for res in result]

print('accuracy:', np.mean(acc), '+/-', np.std(acc))
print('loss:', np.mean(loss), '+/-', np.std(loss))

accuracy: 0.90521973 +/- 0.0052844575
loss: 1.3722885 +/- 0.1595051
